In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.request
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel, BartForConditionalGeneration
import os
import glob
from pathlib import Path


In [ ]:
"""
Scraping articles
"""
# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text
links = ['https://www.tomshardware.com/features/nvidia-ada-lovelace-and-geforce-rtx-40-series-everything-we-know',
         'https://www.tomshardware.com/news/amd-intros-zen-4-ryzen-7000-cpus-and-600-series-chipset-up-to-55-ghz-15-performance-rdna-2-igpus-pcie-5-ddr5',
         'https://www.tomshardware.com/news/intel-core-i9-12900ks-cpu-review',
         'https://www.tomshardware.com/news/nvidia-geforce-rtx-3090-ti-launches-at-1999-dollars',
         'https://www.tomshardware.com/reviews/msi-z690-edge-wifi-ddr4-review',
         'https://www.tomshardware.com/news/amd-expo-memory-oc-feature-confirmed-in-msi-computex-presentation',
         'https://www.tomshardware.com/reviews/lenovo-legion-5i-pro-gen-7',
         'https://www.tomshardware.com/reviews/samsung-t7-shield-portable-ssd-review',
         'https://www.tomshardware.com/reviews/sabrent-rocket-4-plus-8tb-ssd-review',
         'https://www.tomshardware.com/reviews/asus-zenbook-s-13-oled-ryzen-6800u']
for i,link in enumerate(links):
  df=open('article'+str(i)+'.txt','w')
  # providing url
  url = link
  # opening the url for reading
  html = urllib.request.urlopen(url)
  # parsing the html file
  htmlParse = BeautifulSoup(html, 'html.parser')
  # getting all the paragraphs
  for para in htmlParse.find_all("p"):
      df.write( para.get_text())
      df.write('\n')
  df.close()

In [24]:
"""
In order to perform the summarization we need to load the objects:
Tokenizer and the pre trained model which are in our case the gpt-j-6B.
"""
tokenizer  = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")# "EleutherAI/gpt-j-6B"
model      = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")# "EleutherAI/gpt-j-6B"
summarizer = pipeline('summarization',tokenizer=tokenizer,model=model)

In [28]:
"""
Summarization
"""
def return_summary(summarizer):
    articles  = [f for f in glob.glob("*.txt")]
    all_summaries  = open("all_summaries.txt",'w')
    for article in articles:
      try:
        # read the file
        text = Path(article).read_text()
        # call summarizer
        summary_text = summarizer(text,do_sample=False)
        # save sammury
        all_summaries.write(summary_text[0]['summary_text'])
        all_summaries.write('\n')
      except:
        continue
    all_summaries.close()
    """
    Final results
    """
    # read the file
    results_text_input = Path("all_summaries.txt").read_text()
    summary_text_results = summarizer(results_text_input,do_sample=False)
    return summary_text_results[0]['summary_text']

In [31]:
summary = return_summary(summarizer)

Your max_length is set to 142, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
Your max_length is set to 142, but you input_length is only 136. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=68)


In [32]:
print(summary)

 AMD EXPO technology is AMD's alternative to Intel XMP 3.0 . AMD has asked that the slide mentioning EXPO be removed from places where it has been published . MSI revealed that Raphael improves on its predecessor with support for memory overclocking . Visit CNN iReport.com/Travel next Wednesday for a new gallery of snapshots .
